In [168]:
from collections import Counter
import pandas as pd
from datetime import datetime

def check_lemma(lemma, named_entities):
        # mark meaningless entities
        if lemma in ['сми', 'суббота', 'организация', 'правительство', 'посольство', 'мид', 
                    'министерство иностранный дело', 'министерство', 'обратить', 'комиссия', 'вы', 'администрация',
                    'совет', 'министр иностранный дело', 'председатель', 'министр', 'премьер - министр', 'ваш',
                    'юго - восток', 'север', 'миссия', 'рад', 'смид', 'совет министр иностранный дело', 'восток', 
                     'госдепартамент', 'конгресс']:
            return 'no_count'
        
        # replace synonyms with one word
        if (lemma == 'содружество' and '|снг|' in named_entities) or lemma == 'содружество независимый государство':
            return 'снг'
        if 'озхий' in lemma:
            return 'озхо'
        if 'скрипал' in lemma:
            return 'скрипаль'
        synonyms = {
            'атр':['азиатско - тихоокеанский регион'],
            'ссср':['советский союз'],
            'оон':['организация объединить нация'],
            'иг':['исламский государство', 'игил'],
            'сирия':['сара'],
            'евразэс':['евразийский экономический союз'],
            'ес':['евросоюз', 'европейский союз'],
            'шос':['шанхайский организация сотрудничество', 'шанхайский организация сотрудничество ( шос )'],
            'россия':['российский федерация', 'рф'],
            'с.в.лавров':['с.в.лавр'],
            'асеан':['асеана'],
            'астана':['астан'],
            'джабхат ан - нусра':['джабхата ан - нуср'],
            'северный африка':['север африка'],
            'фрг':['германия'],
            'сша':['соединить штат америка', 'соединить штат'],
            'китай':['кнр'],
            'обсе':['организация по безопасность и сотрудничество в европа'],
            'йемен':['йеменский республика ( йр )'],
            'евроатлантика':['евро - атлантика'],
            'африка':['африканский континент'],
            'нато':['североатлантический альянс'],
            'лаг':['лига арабский государство', 'лига арабский государство ( лаг )'],
            'литва':['литовский республика']
        }
        for keyword, synonyms in synonyms.items():
            if lemma in synonyms:
                return keyword
        return 'nan'
    
def clean_named_entities(named_entities):
    if type(named_entities) != float:
        entities = named_entities.split('\n')
        entities_upd = []
        for entity in entities:
            entity_data = entity.split('|')
            lemma = entity_data[3]
            new_lemma = check_lemma(lemma, named_entities)
            if new_lemma == 'nan':
                new_lemma = lemma
            if new_lemma != 'no_count':
                entity_data_upd = '|'.join(entity_data[:3] + [new_lemma] + entity_data[4:])
            else:
                entity_data_upd = 'no_count'
            entities_upd.append(entity_data_upd)
        entities = '\n'.join(entities_upd)
        return entities
    else:
        return float('nan')
    

def sort_by_official_dates(df):
    df1 = pd.DataFrame() # < 2008
    df2 = pd.DataFrame() # 2008 - 2013
    df3 = pd.DataFrame() # 2013 - 2016
    df4 = pd.DataFrame() # > 2016
    date1 = datetime.strptime('15-07-2008', '%d-%m-%Y').date()
    date2 = datetime.strptime('20-02-2013', '%d-%m-%Y').date()
    date3 = datetime.strptime('30-11-2016', '%d-%m-%Y').date()
    for index, row in df.iterrows():
        if int(row['Unnamed: 0']) not in [10995, 11494]:
            try:
                date = datetime.strptime(row['date'].strip(), '%d-%m-%Y').date()
            except ValueError:
                date = datetime.strptime(row['date'], '%m-%Y').date()
            if date <= date1:
                df1 = df1.append(row, ignore_index=True)
            if date > date1 and date <= date2:
                df2 = df2.append(row, ignore_index=True)
            if date > date2 and date <= date3:
                df3 = df3.append(row, ignore_index=True)
            if date > date3:
                df4 = df4.append(row, ignore_index=True)
    return df1, df2, df3, df4

In [25]:
df = pd.read_csv('mfa_texts_df.csv', sep = '\t', encoding = 'utf-8')

In [169]:
# clean named_entities
df['named_entities_preprocessed'] = df['named_entities'].apply(clean_named_entities)

In [177]:
# count most freq entities
def get_top_x_ne(df, x):
    all_entities = []
    for index, row in df.iterrows():
        named_entities = row['named_entities_preprocessed']
        if type(named_entities) != float:
            all_lemmas = []
            named_entities = named_entities.split('\n')
            for named_entity in named_entities:
                if named_entity != 'no_count':
                    all_lemmas.append(named_entity.split('|')[3])
            unique_lemmas = list(set(all_lemmas))
            all_entities.extend(unique_lemmas)

    freq_dict = dict(Counter(all_entities))
    top = sorted(freq_dict.items(), key = lambda kv: kv[1], reverse=True)[:x]
    print(top)
#sorted_dict = sorted(freq_dict.items(), key = lambda kv: kv[1], reverse=True)
#anti_top_20000 = sorted(freq_dict.items(), key = lambda kv: kv[1])[:20000]
#print(anti_top_20000)

# print(sorted_dict[435:500])

In [179]:
get_top_x_ne(df, 100)

[('россия', 9992), ('оон', 4529), ('москва', 3866), ('сша', 3055), ('сирия', 1858), ('с.в.лавров', 1831), ('украина', 1680), ('ес', 1654), ('в.в.путин', 1587), ('совет безопасность', 1397), ('обсе', 1394), ('европа', 1391), ('ближний восток', 1268), ('вашингтон', 976), ('афганистан', 911), ('снг', 882), ('нато', 864), ('ирак', 799), ('киев', 734), ('китай', 710), ('иран', 668), ('д.а.медведев', 660), ('шос', 645), ('франция', 626), ('северный африка', 618), ('фрг', 614), ('ливия', 594), ('иг', 591), ('великобритания', 533), ('совет европа', 523), ('нью - йорк', 522), ('санкт - петербург', 518), ('африка', 513), ('ссср', 510), ('одкб', 508), ('турция', 501), ('атр', 494), ('запад', 493), ('генеральный ассамблея', 476), ('женева', 466), ('дамаск', 453), ('сочи', 446), ('израиль', 434), ('грузия', 396), ('япония', 394), ('казахстан', 374), ('индия', 372), ('брюссель', 367), ('корейский полуостров', 367), ('федеральный собрание', 363), ('латинский америка', 345), ('белоруссия', 343), ('пол

In [180]:
get_top_x_ne(df, 1000)

[('россия', 9992), ('оон', 4529), ('москва', 3866), ('сша', 3055), ('сирия', 1858), ('с.в.лавров', 1831), ('украина', 1680), ('ес', 1654), ('в.в.путин', 1587), ('совет безопасность', 1397), ('обсе', 1394), ('европа', 1391), ('ближний восток', 1268), ('вашингтон', 976), ('афганистан', 911), ('снг', 882), ('нато', 864), ('ирак', 799), ('киев', 734), ('китай', 710), ('иран', 668), ('д.а.медведев', 660), ('шос', 645), ('франция', 626), ('северный африка', 618), ('фрг', 614), ('ливия', 594), ('иг', 591), ('великобритания', 533), ('совет европа', 523), ('нью - йорк', 522), ('санкт - петербург', 518), ('африка', 513), ('ссср', 510), ('одкб', 508), ('турция', 501), ('атр', 494), ('запад', 493), ('генеральный ассамблея', 476), ('женева', 466), ('дамаск', 453), ('сочи', 446), ('израиль', 434), ('грузия', 396), ('япония', 394), ('казахстан', 374), ('индия', 372), ('брюссель', 367), ('корейский полуостров', 367), ('федеральный собрание', 363), ('латинский америка', 345), ('белоруссия', 343), ('пол

In [172]:
sorted_dfs = sort_by_official_dates(df)

In [181]:
get_top_x_ne(sorted_dfs[0], 100) # > 2008

[('россия', 1396), ('оон', 716), ('москва', 591), ('ес', 358), ('в.в.путин', 322), ('сша', 312), ('европа', 301), ('совет безопасность', 249), ('снг', 235), ('ирак', 224), ('обсе', 202), ('ближний восток', 172), ('санкт - петербург', 147), ('афганистан', 146), ('нато', 144), ('совет европа', 143), ('китай', 132), ('иран', 132), ('косово', 129), ('ссср', 116), ('израиль', 110), ('украина', 99), ('атр', 97), ('с.в. лавров', 91), ('шос', 86), ('грузия', 84), ('азия', 84), ('индия', 82), ('нью - йорк', 81), ('франция', 79), ('группа восемь', 79), ('генеральный ассамблея', 76), ('одкб', 76), ('фрг', 71), ('центральный азия', 71), ('ливан', 70), ('африка', 70), ('федеральный собрание', 69), ('с.в.лавров', 65), ('белоруссия', 65), ('казахстан', 65), ('эстония', 64), ('магатэ', 63), ('запад', 63), ('великобритания', 62), ('латвия', 61), ('балканы', 60), ('вто', 60), ('пна', 56), ('лаг', 55), ('корейский полуостров', 54), ('государственный дума', 53), ('нпо', 52), ('евразэс', 51), ('асеан', 50)

In [182]:
get_top_x_ne(sorted_dfs[1], 100) # 2008 - 2013

[('россия', 2694), ('оон', 1031), ('москва', 981), ('сша', 620), ('д.а.медведев', 531), ('с.в.лавров', 485), ('ес', 440), ('совет безопасность', 407), ('обсе', 355), ('европа', 319), ('афганистан', 305), ('ближний восток', 292), ('снг', 266), ('грузия', 214), ('нато', 208), ('сирия', 207), ('южный осетия', 194), ('шос', 188), ('китай', 182), ('украина', 173), ('иран', 166), ('одкб', 161), ('франция', 153), ('фрг', 152), ('атр', 151), ('вашингтон', 146), ('в.в.путин', 146), ('абхазия', 145), ('япония', 145), ('нью - йорк', 145), ('ссср', 133), ('израиль', 131), ('белоруссия', 130), ('казахстан', 128), ('совет европа', 126), ('тбилиси', 121), ('лаг', 121), ('северный африка', 121), ('санкт - петербург', 121), ('женева', 118), ('совет россия - нато', 116), ('ливия', 115), ('евроатлантика', 115), ('корейский полуостров', 109), ('индия', 102), ('египет', 101), ('африка', 98), ('великобритания', 96), ('генеральный ассамблея', 96), ('евразэс', 96), ('брюссель', 94), ('польша', 92), ('закавказ

In [183]:
get_top_x_ne(sorted_dfs[2], 100) # 2013 - 2016

[('россия', 2537), ('оон', 1226), ('москва', 1045), ('сша', 807), ('сирия', 796), ('украина', 787), ('с.в.лавров', 516), ('в.в.путин', 495), ('ближний восток', 442), ('ес', 431), ('обсе', 410), ('европа', 367), ('киев', 365), ('совет безопасность', 351), ('иг', 308), ('северный африка', 293), ('ирак', 257), ('вашингтон', 224), ('ливия', 222), ('нато', 219), ('афганистан', 216), ('дамаск', 192), ('китай', 176), ('женева', 168), ('снг', 166), ('фрг', 163), ('франция', 162), ('шос', 157), ('турция', 155), ('запад', 143), ('африка', 140), ('джабхат ан - нусра', 137), ('брикс', 133), ('брюссель', 128), ('йемен', 124), ('атр', 123), ('алеппо', 122), ('совет европа', 121), ('донбасс', 115), ('нью - йорк', 115), ('генеральный ассамблея', 114), ('крым', 112), ('дж . керри', 109), ('польша', 108), ('великобритания', 108), ('одкб', 108), ('иран', 107), ('санкт - петербург', 99), ('минск', 97), ('сочи', 97), ('индия', 95), ('латинский америка', 95), ('донецкий', 94), ('донецк', 94), ('ссср', 94), 

In [184]:
get_top_x_ne(sorted_dfs[3], 100) # > 2016

[('россия', 3365), ('оон', 1556), ('сша', 1316), ('москва', 1249), ('сирия', 811), ('с.в.лавров', 765), ('в.в.путин', 624), ('украина', 621), ('вашингтон', 559), ('обсе', 427), ('ес', 425), ('европа', 404), ('совет безопасность', 390), ('ближний восток', 362), ('киев', 300), ('нато', 293), ('иг', 283), ('сочи', 279), ('великобритания', 267), ('иран', 263), ('ливия', 254), ('афганистан', 244), ('франция', 232), ('ирак', 230), ('фрг', 228), ('запад', 226), ('турция', 222), ('китай', 220), ('снг', 215), ('шос', 214), ('африка', 205), ('дамаск', 194), ('генеральный ассамблея', 190), ('северный африка', 187), ('нью - йорк', 181), ('ссср', 167), ('еаэс', 163), ('одкб', 163), ('озхо', 162), ('женева', 162), ('венесуэла', 155), ('санкт - петербург', 151), ('брикс', 148), ('лондон', 141), ('астана', 141), ('йемен', 140), ('япония', 138), ('идлиб', 137), ('совет европа', 133), ('корейский полуостров', 132), ('донбасс', 130), ('латинский америка', 126), ('польша', 124), ('федеральный собрание', 1